In [12]:
from igraph import *
import pickle as pkl
import pandas as pd
import numpy as np
import glob
import re
from scipy.spatial.distance import euclidean

In [4]:
path_features = "./extracted_features/"
bboxes_global_classes = "./images/mit67_subclass_bboxesGlobalClasses.txt"
subclass_global_classes = "./images/mit67_subclass_globalClasses.txt"
subclass_bboxes = "./images/mit67_subclass_bboxes.txt"

In [12]:
with open(subclass_bboxes, "r") as f:
    bboxes = pd.read_csv(f, delimiter=" ", usecols=([2]), header=None).to_numpy()
    bboxes = np.unique(bboxes, return_counts=True)
    
bboxes[1]

array([ 65,  63,  58,  57, 196, 350,  64,  22,  42,  20,  38,  55,  16,
        22,  17,  19,  17, 124,  19,  17, 143,  19,  17,  18,  19,  19,
        19,  19,  18,  18,  23,  18, 308,  19,  17, 132,  19,  18,  91,
       108,  19,  17,  18,  16,  19,  19,  17,  17,  19,  17,  17,  53,
        16], dtype=int64)

In [13]:
g = Graph()
g.vs["name"] = None

In [14]:
for name, count in zip(bboxes[0], bboxes[1]):
      
    g1 = Graph.Full(count)
    for i in range(count):
        g1.vs[i]["name"] = name + "_" + str(i)
    
    names = g.vs["name"] + g1.vs["name"]
    g = g + g1
    g.vs["name"] = names
    
g.summary()

'IGRAPH UN-- 2607 183252 -- \n+ attr: name (v)'

In [15]:
g.vs["name"]

['airport_inside_0',
 'airport_inside_1',
 'airport_inside_2',
 'airport_inside_3',
 'airport_inside_4',
 'airport_inside_5',
 'airport_inside_6',
 'airport_inside_7',
 'airport_inside_8',
 'airport_inside_9',
 'airport_inside_10',
 'airport_inside_11',
 'airport_inside_12',
 'airport_inside_13',
 'airport_inside_14',
 'airport_inside_15',
 'airport_inside_16',
 'airport_inside_17',
 'airport_inside_18',
 'airport_inside_19',
 'airport_inside_20',
 'airport_inside_21',
 'airport_inside_22',
 'airport_inside_23',
 'airport_inside_24',
 'airport_inside_25',
 'airport_inside_26',
 'airport_inside_27',
 'airport_inside_28',
 'airport_inside_29',
 'airport_inside_30',
 'airport_inside_31',
 'airport_inside_32',
 'airport_inside_33',
 'airport_inside_34',
 'airport_inside_35',
 'airport_inside_36',
 'airport_inside_37',
 'airport_inside_38',
 'airport_inside_39',
 'airport_inside_40',
 'airport_inside_41',
 'airport_inside_42',
 'airport_inside_43',
 'airport_inside_44',
 'airport_inside_45'

In [16]:
def return_features(data):
    if type(data) == type(np.array(0.0)):
        return data
    return None

In [17]:
def return_names(data):
    if type(data) == type("string"):
        return data

In [18]:
def counter(data):
    if type(data) == type(np.array(0.0)):
        return 1
    return 0

In [20]:
count = 0
previous_name = ""
files = sorted(glob.glob(path_features+"*"), key=sortKeyFunc)
total = len(files)
for indice, file in enumerate(files):
    
    df = pd.read_pickle(file)
    sys.stdout.write("Dataframes lidos: %f %%  \r" % ((indice/total)*100) )
    sys.stdout.flush()
    
    for c in range(len(df)):
        length = np.sum(df.iloc[c].apply(counter))
#         if length == 0:
            
        g1 = Graph.Full(length)

        name = df.iloc[c].global_class
        
        if previous_name != name:
            count = 0
        
        for i in range(length):
            g1.vs[i]["name"] = name + "_" + str(count) + "_" + str(i)
        
        names = g.vs["name"] + g1.vs["name"]
        g = g + g1
        g.vs["name"] = names
        
        global_indice = g.vs.find(name=name + "_" + str(count)).index
        for i in range(length):
            bbox_indice = g.vs.find(name=name + "_" + str(count) + "_" + str(i)).index
            g.add_edge(global_indice, bbox_indice)
        
        count += 1
        previous_name = name

In [22]:
g.summary()

'IGRAPH UN-- 56082 959809 -- \n+ attr: name (v)'

In [23]:
count = 0
previous_name = ""
for file in files:
    df = pd.read_pickle(file)
    
    for c in range(len(df)):
        bbox_features = df.iloc[c].apply(return_features).dropna()
        length = len(bbox_features)
        
        name = df.iloc[c].global_class
        
        if previous_name != name:
            count = 0
        
        global_indice = g.vs.find(name=name + "_" + str(count)).index
        g.vs[global_indice]["feature"] = df.iloc[c].global_feature
        
        for i in range(length):
            bbox_indice = g.vs.find(name=name + "_" + str(count) + "_" + str(i)).index
            g.vs[bbox_indice]["feature"] = bbox_features[i]
            
        count += 1
        previous_name = name

In [24]:
for indice, pair in enumerate(g.get_edgelist()):
    features = g.vs[pair]["feature"]
    g.es[indice]["weight"] = euclidean(features[0].reshape(-1), features[1].reshape(-1))

In [25]:
mst_g = g.spanning_tree(weights=g.es["weight"])

In [26]:
mst_g.summary()

'IGRAPH UNW- 56082 56029 -- \n+ attr: feature (v), name (v), weight (e)'

In [32]:
with open("graph.pkl", "wb") as f:
    mst_g.write_pickle(f)